In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-06-28 17:41:10--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2025-06-28 17:41:10 (110 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

In [ ]:
print(text[:1000])

In [ ]:
!pip install nltk

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
# Tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
# tokenize
tokens = word_tokenize(text.lower())

In [ ]:
vocab = {'<unk>':0}
for token in Counter(tokens).keys():
    if token not in vocab:
        vocab[token] = len(vocab)

In [ ]:
len(vocab)

In [ ]:
input_sentences = text.split('\n')

In [ ]:
len(input_sentences)

In [ ]:
input_sentences = [s for s in input_sentences if s != ""]

In [ ]:
merged_sentences = []

# pick how many lines to merge — you said 4–5, so we’ll do 5
chunk_size = 5

# make sure your original list has no empty lines
input_sentences = [s.strip() for s in input_sentences if s.strip()]

# loop in steps of `chunk_size`
for i in range(0, len(input_sentences), chunk_size):
    chunk = input_sentences[i:i+chunk_size]
    merged = ' '.join(chunk)
    merged_sentences.append(merged)

print(merged_sentences[:3])  # preview first 3 chunks

In [ ]:
len(merged_sentences)

In [ ]:
merged_sentences[:5]

In [ ]:
def text_to_indices(sentence, vocab):

  numerical_sentence = []

  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence

In [ ]:
input_numerical_sentences = []

for sentence in merged_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))

In [ ]:
len(input_numerical_sentences)

In [ ]:
training_sequence = []
for sentence in input_numerical_sentences:

  for i in range(1, len(sentence)):
    training_sequence.append(sentence[:i+1])

In [ ]:
len(training_sequence)

In [ ]:
training_sequence[:9]

In [ ]:
max_len_list =0

for sequence in training_sequence:
  if len(sequence) >= max_len_list:
      max_len_list= len(sequence)

max_len_list

In [ ]:
training_sequence[0]

In [ ]:
padded_training_sequence = []

for sequence in training_sequence:

  padded_training_sequence.append([0]*(max_len_list - len(sequence)) + sequence)

In [ ]:
padded_training_sequence[:3]

In [ ]:
len(padded_training_sequence[10])

In [ ]:
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)

In [ ]:
padded_training_sequence[:5,:]

In [ ]:
X = padded_training_sequence[:, :-1]
y = padded_training_sequence[:,-1]

In [ ]:
X[:3,:], y[:3]

In [ ]:
class CustomDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [ ]:
dataset = CustomDataset(X,y)
len(dataset)

In [ ]:
dataloader = DataLoader(dataset, batch_size=2048, pin_memory=True, shuffle=True)

In [ ]:
dataloader

In [ ]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim=256, hidden_dim=256, num_layers=2, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(
            input_size=embed_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, vocab_size)
        self.layer_norm = nn.LayerNorm(hidden_dim)

    def forward(self, x):
        embedded = self.embedding(x)                     # (batch, seq_len, embed_dim)
        lstm_out, (hn, cn) = self.lstm(embedded)         # hn: (num_layers, batch, hidden_dim)
        last_hidden = hn[-1]                             # get the last layer's hidden state -> (batch, hidden_dim)
        normed = self.layer_norm(last_hidden)            # layer norm
        dropped = self.dropout(normed)                   # dropout
        output = self.fc(dropped)                        # final output: (batch, vocab_size)
        return output

In [ ]:
model = LSTMModel(len(vocab))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model.to(device)

In [ ]:
device

In [ ]:
epochs = 50
learning_rate = 0.005

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# training loop

for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in dataloader:

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")

In [ ]:
def generate_live(model, vocab, idx_to_word, word_to_idx, start_text, device, length=50, temperature=1.0):
    model.eval()
    
    tokens = word_tokenize(start_text.lower())
    input_idx = [word_to_idx[word] for word in tokens if word in word_to_idx]
    input_tensor = torch.tensor([input_idx], dtype=torch.long, device=device)

    with torch.no_grad():
        embedded = model.embedding(input_tensor)
        output, (h, c) = model.lstm(embedded)

    current_idx = input_idx[-1]
    generated = tokens.copy()

    sys.stdout.write(' '.join(generated) + ' ')
    sys.stdout.flush()

    for _ in range(length):
        input_tensor = torch.tensor([[current_idx]], dtype=torch.long, device=device)
        embedded = model.embedding(input_tensor)
        output, (h, c) = model.lstm(embedded, (h, c))
        logits = model.fc(output.squeeze(1))
        logits = logits / temperature
        probs = torch.softmax(logits, dim=-1)
        next_idx = torch.multinomial(probs, num_samples=1).item()
        next_word = idx_to_word[next_idx]

        sys.stdout.write(next_word + ' ')
        sys.stdout.flush()

        current_idx = next_idx

        time.sleep(0.1)  # if you want live typing effect

    print()  # new line at end

In [ ]:
generate_live(
    model,
    vocab=vocab,
    idx_to_word=list(vocab.keys()),
    word_to_idx=vocab,
    start_text="i ",
    device=device,
    length=200,
    temperature=0.8
)

In [ ]:
torch.save(model.state_dict(), 'lstm_shakespeare.pth')

In [ ]:
model.load_state_dict(torch.load('lstm_shakespeare.pth'))

In [ ]:
model.eval()